# Regression Analysis

## Does crative features influence on Click Through Rate?

-- Cuáles son los drivers de performance en Paid Social (TikTok)?

In [25]:
# Data manipulation libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling libraries
import statsmodels.api as sm

In [26]:
df = pd.read_csv('../../data/datasets/tiktok_topads_clean.csv')
df.head()

,ad_title,brand_name,cost,ctr,id,like,is_ugc,duration,height,width,industry_parent,industry_child,objective_value,creative_theme,creative_concept,format_production_style,talent_type,demographic_representation,audience_focus,campaign_objective
0,Great time killer!,Survival Game Master,2,0.01,7132878852058906625,38213,True,30.974,1280.0,720.0,News & Entertainment,Games & Utility Software,App Installs,Promotional & Offer-Based,Product demo,Native Video,Influencers,Primarily Male,Unaware Audience,App Promotion
1,Oddly satisfying game,Gameworld Master,2,0.01,7109275920046178305,14299,False,37.334,1024.0,576.0,News & Entertainment,Games & Utility Software,App Installs,Product-Centric,Product demo,Native Video,NaN,No People Featured,Unaware Audience,Awareness
2,My friend recommended me to play this game,Sugarcane Inc. Empire Tycoon,0,0.02,7077499601561305089,2427,False,17.323,1024.0,576.0,News & Entertainment,Games & Utility Software,App Installs,Educational & Explainer,Product demo,Animation & Motion Graphics,NaN,No People Featured,Unaware Audience,Awareness
3,download now,"Smart VPN - Fast, Stable",2,0.02,7176836203668783106,124771,False,53.015,1280.0,720.0,News & Entertainment,Games & Utility Software,App Installs,Lifestyle & Aspirational,Before-and-after story,Animation & Motion Graphics,NaN,Primarily Female,Unclear,Engagement
4,Play ten minutes a day to relieve stress!,Sugarcane Factory 3D,0,0.02,7077499507373899777,2739,True,14.016,1024.0,576.0,News & Entertainment,Games & Utility Software,App Installs,Product-Centric,Product demo,Animation & Motion Graphics,NaN,No People Featured,Unaware Audience,App Promotion


## Train the linear regression

In [344]:
#lets see if there are any non numeric features
print(df.select_dtypes(include='object').columns)
print(df.select_dtypes(include='bool').columns)


Index([], dtype='object')
Index(['is_ugc', 'industry_parent_Apparel & Accessories',
       'industry_parent_Appliances', 'industry_parent_Apps',
       'industry_parent_Baby, Kids & Maternity',
       'industry_parent_Beauty & Personal Care',
       'industry_parent_Business Services',
       'industry_parent_E-Commerce (Non-app)', 'industry_parent_Education',
       'industry_parent_Financial Services',
       ...
       'audience_focus_Solution Aware', 'audience_focus_Unaware Audience',
       'audience_focus_Unclear', 'campaign_objective_App Promotion',
       'campaign_objective_Awareness', 'campaign_objective_Engagement',
       'campaign_objective_Leads', 'campaign_objective_Sales',
       'campaign_objective_Traffic', 'campaign_objective_Unclear'],
      dtype='object', length=202)


In [345]:
#I need to cast boolean features to int
df = df * 1

In [346]:
# Prepare X (features) and y (target)
y = df['ctr']
X = df.drop(['ctr', 'cost', 'like'], axis=1)

In [347]:
X.dtypes


is_ugc                                     int64
duration                                 float64
height                                   float64
width                                    float64
industry_parent_Apparel & Accessories      int64
                                          ...   
campaign_objective_Engagement              int64
campaign_objective_Leads                   int64
campaign_objective_Sales                   int64
campaign_objective_Traffic                 int64
campaign_objective_Unclear                 int64
Length: 205, dtype: object

In [348]:
# Add constant for intercept
X = sm.add_constant(X)

# Fit linear regression model
model = sm.OLS(y, X).fit()

# Print model summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.763
Model:                            OLS   Adj. R-squared:                  0.589
Method:                 Least Squares   F-statistic:                     4.376
Date:                Fri, 16 May 2025   Prob (F-statistic):           1.13e-25
Time:                        17:14:30   Log-Likelihood:                 1864.3
No. Observations:                 414   AIC:                            -3377.
Df Residuals:                     238   BIC:                            -2668.
Df Model:                         175                                         
Covariance Type:            nonrobust                                         
                                                                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------